In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, prange
import cv2
import os
import time

# ==========================================
# 1. CONFIGURATION
# ==========================================

# Resolution & Output
WIDTH = 1000
HEIGHT = 750
OUTPUT_DIR = "Kerr_frames_HR"
OUTPUT_VIDEO = "kerr_black_hole_HR_orbit.mp4"

# Physics Parameters
FOV = 40.0           # Slightly tighter FOV for the larger distance
SPIN_A = 0.99        # Near-extremal spin (0.99)
RS = 1.0
# Event Horizon radius for Kerr
R_HORIZON = RS + np.sqrt(RS**2 - SPIN_A**2)

# Integration Settings
STEP_SIZE = 0.05
MAX_STEPS = 3000     # Increased for high precision

# Animation Settings
NUM_FRAMES = 360
FPS = 30
CAM_DIST = 44.0      # Increased distance
ORBIT_INCLINATION = 30.0 # +/- 30 degrees dip

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==========================================
# 2. NUMBA-ACCELERATED PHYSICS KERNELS
# ==========================================

@njit
def get_inverse_metric(pos):
    """
    Returns the inverse metric components g^uv at position (r, theta).
    Boyer-Lindquist coordinates.
    """
    r = pos[1]
    theta = pos[2]

    sin_th = np.sin(theta)
    cos_th = np.cos(theta)
    sigma = r**2 + (SPIN_A * cos_th)**2
    delta = r**2 - 2.0 * RS * r + SPIN_A**2

    # Inverse Metric Components
    inv_g_tt = - ((r**2 + SPIN_A**2)**2 - delta * (SPIN_A * sin_th)**2) / (delta * sigma)
    inv_g_rr = delta / sigma
    inv_g_thth = 1.0 / sigma

    # Polar singularity safeguard
    sin_sq = sin_th**2
    if sin_sq < 1e-9: sin_sq = 1e-9

    inv_g_phiphi = (delta - SPIN_A**2 * sin_sq) / (delta * sigma * sin_sq)
    inv_g_tphi = - (2.0 * RS * r * SPIN_A) / (delta * sigma)

    return inv_g_tt, inv_g_rr, inv_g_thth, inv_g_phiphi, inv_g_tphi

@njit
def hamiltonian(pos, p):
    g_tt, g_rr, g_thth, g_ph, g_tph = get_inverse_metric(pos)
    H = 0.5 * (g_tt * p[0]**2 +
               g_rr * p[1]**2 +
               g_thth * p[2]**2 +
               g_ph * p[3]**2 +
               2.0 * g_tph * p[0] * p[3])
    return H

@njit
def get_derivatives(state):
    pos = state[:4]
    p = state[4:]
    g_tt, g_rr, g_thth, g_ph, g_tph = get_inverse_metric(pos)

    dt  = g_tt * p[0] + g_tph * p[3]
    dr  = g_rr * p[1]
    dth = g_thth * p[2]
    dph = g_ph * p[3] + g_tph * p[0]

    # Numerical differentiation for forces
    epsilon = 1e-5

    # dH/dr
    pos_plus = np.array([pos[0], pos[1] + epsilon, pos[2], pos[3]])
    pos_minus = np.array([pos[0], pos[1] - epsilon, pos[2], pos[3]])
    dH_dr = (hamiltonian(pos_plus, p) - hamiltonian(pos_minus, p)) / (2.0 * epsilon)

    # dH/dtheta
    pos_plus[1] = pos[1]
    pos_minus[1] = pos[1]
    pos_plus[2] = pos[2] + epsilon
    pos_minus[2] = pos[2] - epsilon
    dH_dth = (hamiltonian(pos_plus, p) - hamiltonian(pos_minus, p)) / (2.0 * epsilon)

    dpt = 0.0
    dpphi = 0.0
    dpr = -dH_dr
    dpth = -dH_dth

    return np.array([dt, dr, dth, dph, dpt, dpr, dpth, dpphi])

@njit
def rk4_step_kerr(state, h):
    k1 = get_derivatives(state)
    k2 = get_derivatives(state + 0.5 * h * k1)
    k3 = get_derivatives(state + 0.5 * h * k2)
    k4 = get_derivatives(state + h * k3)
    return state + (h / 6.0) * (k1 + 2*k2 + 2*k3 + k4)

@njit
def get_disk_color(r, x_hit, y_hit):
    """
    Procedural accretion disk texture.
    Using Cartesian (x,y) for pattern to avoid 'atan2' branch cut artifacts.
    """
    # Create pattern using Cartesian coords to ensure continuity
    # Spiral arms: sin(log(r)*A + angle)
    # Angle can be computed as arctan2, but we must be careful.
    # Instead, we use a noise function based on x,y which has no singularity.

    angle = np.arctan2(y_hit, x_hit)
    scale = 3.0

    # Spiral + Rings
    # Adding a small rotation to the spiral based on radius to simulate differential rotation
    spiral = np.sin(r * scale + angle * 2.0)
    rings = np.sin(r * 5.0)

    # Base brightness
    brightness = 0.5 + 0.25 * spiral + 0.25 * rings

    # Doppler Beaming (Relativistic boosting)
    # Approaching side (x < 0 usually) is brighter
    beaming = 1.0 - 0.5 * (x_hit / (r + 0.1))
    brightness *= beaming

    # Temperature Gradient (Hotter near center)
    norm_r = (r - R_HORIZON) / 15.0
    col_r = 1.0 * brightness
    col_g = (0.3 + 0.5 * norm_r) * brightness
    col_b = (0.1 + 0.8 * norm_r) * brightness

    return np.array([col_r, col_g, col_b])

@njit
def ray_march_kerr(cam_pos_bl, ray_p_bl):
    state = np.concatenate((cam_pos_bl, ray_p_bl))

    for i in range(MAX_STEPS):
        prev_r = state[1]
        prev_th = state[2]

        # --- Adaptive Step Size ---
        # 1. Slow down near Event Horizon
        if state[1] < 2.0 * RS:
            h_factor = 0.2
        # 2. Slow down near Polar Singularity to prevent artifacts
        elif np.abs(np.sin(state[2])) < 0.15:
            h_factor = 0.1
        else:
            h_factor = 1.0

        current_h = STEP_SIZE * h_factor

        state = rk4_step_kerr(state, current_h)

        r = state[1]
        th = state[2]

        # Horizon Check
        if r < R_HORIZON * 1.05:
            return np.array([0.0, 0.0, 0.0])

        # Escape Check
        if r > CAM_DIST * 1.5:
             # Starfield background (simple noise based on direction)
             # Use phi and theta to map stars
             phi = state[3]
             stars = np.sin(phi * 30.0) * np.sin(th * 30.0)
             if stars > 0.98: return np.array([1.0, 1.0, 1.0])
             return np.array([0.0, 0.0, 0.02])

        # Disk Intersection Check (Equatorial Plane theta = pi/2)
        # Check if we crossed pi/2
        if (prev_th - np.pi/2.0) * (th - np.pi/2.0) < 0.0:
            # Linear interpolation for exact hit radius
            frac = abs(prev_th - np.pi/2.0) / (abs(prev_th - np.pi/2.0) + abs(th - np.pi/2.0))
            hit_r = prev_r + (r - prev_r) * frac

            # Get exact phi for texture
            prev_phi = state[3] - (state[3] - state[7]*current_h) # Approx prev phi
            # Note: state[3] is current phi.
            # We need to be careful interpolating phi if it wrapped around 2pi.
            # However, for texture generation, we convert to Cartesian immediately.
            hit_phi = state[3] # Approximate is fine for texture noise

            if 2.5 * RS < hit_r < 18.0 * RS:
                # Convert to Cartesian on the disk plane for texture lookup
                x_h = hit_r * np.cos(hit_phi)
                y_h = hit_r * np.sin(hit_phi)
                return get_disk_color(hit_r, x_h, y_h)

    return np.array([0.0, 0.0, 0.0])

@njit(parallel=True)
def render_frame_data(width, height, cam_r, cam_theta, cam_phi, fov):
    image = np.zeros((height, width, 3))
    aspect = width / height

    # 1. Camera Position in Global Cartesian
    cx = cam_r * np.sin(cam_theta) * np.cos(cam_phi)
    cy = cam_r * np.sin(cam_theta) * np.sin(cam_phi)
    cz = cam_r * np.cos(cam_theta)
    cam_pos_cart = np.array([cx, cy, cz])

    # 2. Camera Basis Vectors
    # Look at origin
    fwd = -cam_pos_cart / np.linalg.norm(cam_pos_cart)

    # Robust Up vector
    # If looking straight down/up (Z-axis), change Up reference
    glob_up = np.array([0.0, 0.0, 1.0])
    if np.abs(np.dot(fwd, glob_up)) > 0.95:
        glob_up = np.array([0.0, 1.0, 0.0])

    right = np.cross(fwd, glob_up)
    right = right / np.linalg.norm(right)
    up = np.cross(right, fwd)

    # Screen dimensions
    fov_rad = np.radians(fov)
    half_h = np.tan(fov_rad/2.0)
    half_w = half_h * aspect

    for y in prange(height):
        for x in prange(width):
            sx = (x / width) * 2.0 - 1.0
            sy = 1.0 - (y / height) * 2.0

            # Ray Direction in Cartesian
            ray_dir_cart = fwd + right * (sx * half_w) + up * (sy * half_h)
            ray_dir_cart = ray_dir_cart / np.linalg.norm(ray_dir_cart)

            # Convert to Boyer-Lindquist Initial State
            init_pos = np.array([0.0, cam_r, cam_theta, cam_phi])

            # Convert Momentum (Cartesian -> Spherical)
            vx, vy, vz = ray_dir_cart[0], ray_dir_cart[1], ray_dir_cart[2]

            # Unit vectors for spherical coords at camera position
            # r_hat
            sin_th = np.sin(cam_theta)
            cos_th = np.cos(cam_theta)
            sin_ph = np.sin(cam_phi)
            cos_ph = np.cos(cam_phi)

            # Dot products to project Cartesian velocity v onto spherical basis
            v_dot_rhat = vx*sin_th*cos_ph + vy*sin_th*sin_ph + vz*cos_th
            v_dot_thhat = vx*cos_th*cos_ph + vy*cos_th*sin_ph - vz*sin_th
            v_dot_phhat = -vx*sin_ph + vy*cos_ph

            # Canonical Momenta
            # p_r = v_r
            # p_theta = r * v_theta
            # p_phi = r * sin(theta) * v_phi
            pr = v_dot_rhat
            pth = cam_r * v_dot_thhat
            pph = cam_r * sin_th * v_dot_phhat

            init_p = np.array([-1.0, pr, pth, pph])

            image[y, x] = ray_march_kerr(init_pos, init_p)

    return image

# ==========================================
# 3. ORBIT LOGIC & EXECUTION
# ==========================================

def get_orbit_position(frame_idx, total_frames, distance, inclination_deg):
    """
    Calculates Camera (r, theta, phi) for an INCLINED orbit.
    This varies theta and phi simultaneously.
    """
    # Orbital angle (0 to 2pi)
    alpha = (2.0 * np.pi * frame_idx) / total_frames

    # Inclination (tilt around Y-axis, for example)
    inc_rad = np.radians(inclination_deg)

    # Parametric circle in 3D, tilted
    # Start with circle in X-Z plane (equatorial if Z is up? No, Kerr Z is axis of rotation)
    # Standard Kerr: Z is rotation axis. Equatorial plane is X-Y.
    # We want a circle that goes above and below X-Y.

    # 1. Circle in X-Y plane
    # x = d * cos(alpha)
    # y = d * sin(alpha)
    # z = 0

    # 2. Tilt it. Let's rotate around the X-axis by inclination angle.
    # New Y' = y * cos(inc) - z * sin(inc)
    # New Z' = y * sin(inc) + z * cos(inc)

    # Wait, if we want to orbit the black hole, we basically want to traverse phi.
    # Standard orbit: theta = 90 (equator).
    # Inclined orbit: theta oscillates between 90-inc and 90+inc.

    # Let's define the position vector in Cartesian, then convert to Spherical.
    # Orbit in a plane tilted by 'inc' relative to the equator.
    x_orbit = distance * np.cos(alpha)
    y_orbit = distance * np.sin(alpha) * np.cos(inc_rad)
    z_orbit = distance * np.sin(alpha) * np.sin(inc_rad)

    # Convert to r, theta, phi
    r = np.sqrt(x_orbit**2 + y_orbit**2 + z_orbit**2)
    theta = np.arccos(z_orbit / r)
    phi = np.arctan2(y_orbit, x_orbit)

    return r, theta, phi

def main():
    print("Initializing High-Res Kerr Orbit Simulation...")
    print(f"Output Directory: {OUTPUT_DIR}")
    print(f"Distance: {CAM_DIST}")
    print(f"Inclination: {ORBIT_INCLINATION} deg")

    # JIT Compile (Warmup)
    print("Compiling JIT kernels (this takes a moment)...")
    _ = render_frame_data(10, 10, 20.0, 1.5, 0.0, 60.0)
    print("Compilation Complete.")

    print(f"Rendering {NUM_FRAMES} frames...")

    for i in range(NUM_FRAMES):
        filename = f"{OUTPUT_DIR}/frame_{i:03d}.png"

        if os.path.exists(filename):
            print(f"Frame {i} exists.")
            continue

        # Get Dynamic Camera Position
        cam_r, cam_theta, cam_phi = get_orbit_position(i, NUM_FRAMES, CAM_DIST, ORBIT_INCLINATION)

        t0 = time.time()
        img_data = render_frame_data(WIDTH, HEIGHT, cam_r, cam_theta, cam_phi, FOV)

        # Process Image
        img_data = np.clip(img_data, 0, 1) * 255.0
        img_bgr = img_data.astype(np.uint8)[..., ::-1] # RGB to BGR for OpenCV

        cv2.imwrite(filename, img_bgr)

        elapsed = time.time() - t0
        print(f"Frame {i+1}/{NUM_FRAMES} | Inc: {90 - np.degrees(cam_theta):.1f} deg | Time: {elapsed:.2f}s")

    # Compile Video
    print("Stitching video...")
    images = [img for img in sorted(os.listdir(OUTPUT_DIR)) if img.endswith(".png")]
    if not images: return

    frame_path = os.path.join(OUTPUT_DIR, images[0])
    frame = cv2.imread(frame_path)
    h, w, _ = frame.shape

    video = cv2.VideoWriter(OUTPUT_VIDEO, cv2.VideoWriter_fourcc(*'mp4v'), FPS, (w, h))

    for image in images:
        video.write(cv2.imread(os.path.join(OUTPUT_DIR, image)))

    video.release()
    print(f"Done! Video saved to {OUTPUT_VIDEO}")

if __name__ == "__main__":
    main()

Initializing High-Res Kerr Orbit Simulation...
Output Directory: Kerr_frames_HR
Distance: 44.0
Inclination: 30.0 deg
Compiling JIT kernels (this takes a moment)...
Compilation Complete.
Rendering 360 frames...
Frame 1/360 | Inc: 0.0 deg | Time: 129.25s
Frame 2/360 | Inc: 0.5 deg | Time: 129.73s
Frame 3/360 | Inc: 1.0 deg | Time: 132.91s
Frame 4/360 | Inc: 1.5 deg | Time: 129.15s
Frame 5/360 | Inc: 2.0 deg | Time: 129.97s
Frame 6/360 | Inc: 2.5 deg | Time: 128.63s
Frame 7/360 | Inc: 3.0 deg | Time: 128.42s
Frame 8/360 | Inc: 3.5 deg | Time: 128.70s
Frame 9/360 | Inc: 4.0 deg | Time: 127.74s
Frame 10/360 | Inc: 4.5 deg | Time: 127.52s
Frame 11/360 | Inc: 5.0 deg | Time: 127.04s
Frame 12/360 | Inc: 5.5 deg | Time: 127.36s
Frame 13/360 | Inc: 6.0 deg | Time: 126.97s
Frame 14/360 | Inc: 6.5 deg | Time: 133.92s
Frame 15/360 | Inc: 6.9 deg | Time: 126.78s
Frame 16/360 | Inc: 7.4 deg | Time: 125.86s
Frame 17/360 | Inc: 7.9 deg | Time: 125.73s
Frame 18/360 | Inc: 8.4 deg | Time: 125.74s
Frame 1